# OpenAI Simple Chat Model

In [ ]:
! pip3 install openai

In [ ]:
OPENAI_API_KEY = ""

In [2]:
from agentquest.drivers.mastermind import MasterMindDriver
from agentquest.utils import Action, load_data
from termcolor import cprint

Load a generic 4 digits mastermind game

In [3]:
game = load_data('mastermind', '4 digits')[0]

Initialize the prompt and the LLM

In [4]:
from openai import OpenAI

prompt = '''You are tasked to play the Mastermind game. The host chooses a 
number and gives you the amount of digits. You have to gess the correct number
as fast as you can.
The number can contains repetitions.
The number can contain any possible digit between:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9

At each round you provide a number as guess. The hosts provides you 
1. The number of correct digits in the wrong position
2. The number of correct digits in the correct position

The game ends when the hosts outputs 'You Won!'

Carefully choose your strategy. Avoid bruteforce.

Use the following format:
THOUGHT: think about your guess
ACTION: only your guessed number without additional info.
'''

# Initialize the LLM
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

# Initalize the system prompt
messages=[
    {"role": "system", "content": prompt},
]

Initialize the mastermind driver and get the first observation

In [5]:
driver = MasterMindDriver(game)
obs = driver.reset() # Get the first observation
print(f'OBSERVATION: {obs.output}')

OBSERVATION: Start guessing the 4 digits number.


Start the LLM loop solving the task

In [6]:
step_cnt = 0
while not obs.done and step_cnt<10:
    # Provide the observation to the LLM
    messages.append({"role": "user", "content": obs.output})
    
    # Get the LLM output
    agent_choice = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    ).choices[0].message.content
    print(agent_choice)
    
    # Provide the choice to the LLM as history
    messages.append({"role": "assistant", "content": agent_choice})
    
    # Extract the LLM guess
    guess = agent_choice.split('ACTION: ')[-1]
    
    # Build the action
    action = Action(action_value=guess)
    
    # Run a mastermind round and get the observation
    obs = driver.step(action)
    print(f'OBSERVATION: {obs.output}')

    step_cnt+=1

THOUGHT: Since I don't have any information about the number yet, I will start with a random guess.

ACTION: 1234
OBSERVATION: Your guess has 1 correct numbers in the wrong position and 0 correct numbers in the correct position. Keep guessing.
THOUGHT: Since I have one correct number in the wrong position, I can use this information to narrow down the possibilities for the correct number.

ACTION: Let's try a guess where one digit is in the correct position and the other three are different.

ACTION: 1023
OBSERVATION: Your guess has 1 correct numbers in the wrong position and 0 correct numbers in the correct position. Keep guessing.
THOUGHT: The information from the previous guess confirms that the digit '1' is in the correct position. I will try to find the other digits that might be in the correct position.

ACTION: Let's try a guess where the digits '1' and '3' are in the correct positions and the other two digits are different.

ACTION: 1302
OBSERVATION: Your guess has 1 correct nu